In [6]:
# from sklearn.datasets import make_multilabel_classification
from torch.utils.data import Dataset
import json
import numpy as np
import pandas as pd
import torch
import time
from tqdm import tqdm

In [7]:
# f = open("outputs/year_venue.json", 'r')
f = open("outputs/year_venue_naive_bayes.json", 'r')
year_venue = json.load(f)

f = open("outputs/abstracts_title.json", 'r')
abstracts_title = json.load(f)

f = open("outputs/author.json", 'r')
author = json.load(f)


In [8]:
#################################
w = 1  # abstracts - title
a = 1  # authors
y = 1  # year - venue
sum_ = w+a+y
#################################

YEAR_VENUE_WEIGHT = y/sum_
COAUTHOR_WEIGHT = a/sum_
SENTENCE_WEIGHT = w/sum_

def get_weighted_value(i, j):
    val1 = year_venue[i][j] * YEAR_VENUE_WEIGHT
    val2 = author[i][j] * COAUTHOR_WEIGHT
    val3 = abstracts_title[i][j] * SENTENCE_WEIGHT
    
    return val1 + val2 + val3
    

In [12]:
THRESHOLD = 0.2

kaggle_predict = []

for i in range(800):
    
    pred = ""
    
    for j in range(100):
        
        val = get_weighted_value(str(i), str(j))
        
        if val >= THRESHOLD:
            pred += str(j) + " "
#             print(pred)
            
    if pred:
        kaggle_predict.append(pred[:-1])
    else:
        kaggle_predict.append("-1")

        

In [13]:
kaggle = pd.DataFrame(columns=["ID", "Predict"])
kaggle['Predict'] = kaggle_predict
kaggle['ID'] = kaggle.index
kaggle = kaggle.set_index('ID')
kaggle.to_csv("../../kaggle/predict_stacking_10_3.csv")

In [14]:
count = 0
for i in kaggle['Predict']:
    if i != '-1':
        count += 1

count

544

# Kaggle Predictions

- YEAR_VENUE_WEIGHT = 1/3
- COAUTHOR_WEIGHT = 1/3
- SENTENCE_WEIGHT = 1/3
    - f1 = 0.47230

# Kaggle Predictions - 2

- YEAR_VENUE_WEIGHT = 0.08884680485159918.
- COAUTHOR_WEIGHT = 0.5925861815691253
- SENTENCE_WEIGHT = 0.3185670135792754
    - f1 = 0.52080

In [49]:
print(YEAR_VENUE_WEIGHT)
print(COAUTHOR_WEIGHT)
print(SENTENCE_WEIGHT)

0.09421771031395267
0.43393907367832263
0.4718432160077247


# Kaggle Predictions - 3

- YEAR_VENUE_WEIGHT = 0.09421771031395267.
- COAUTHOR_WEIGHT = 0.43393907367832263
- SENTENCE_WEIGHT = 0.4718432160077247
- THRESHOLD = 0.3
    - f1 = 0.53130

In [43]:
a = [0.9566845690739418,
 0.9977403210423819,
 0.9990949067782368,
 0.9995765717884457,
 0.9996834779248579,
 0.9996834779248579,
 0.9997960250953476,
 0.9997906369281058,
 0.9997804862444465]

b = [0.9140499814085832,
 0.9172208109372708,
 0.9194803406690103,
 0.9028562145490067,
 0.9062611898144489,
 0.9100093884489224,
 0.9120535730073112,
 0.9109805733621669,
 0.9083259223343757]

c = [0.03217569098314048,
 0.033274246786204,
 0.09689634519674081,
 0.1266490732680644,
 0.18306285315357687,
 0.19963939094534452,
 0.1829821136259908,
 0.15826426166468657,
 0.1324441261793674]

In [46]:
max_ = 0
idx = 0
for i in range(len(a)):
    if a[i] + b[i] + c[i] >= max_:
        max_ = a[i] + b[i] + c[i]
        idx = i

In [48]:
thresholds = [0.01, 0.02, 0.05, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6]
thresholds[5]

0.3